#### Take rate vs. days after cancel

In [0]:
sql
with empty_rows as (
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n
  )
  
      select row_number() over(partition by e1.n order by e1.n) days
      from empty_rows e1
      join empty_rows e2
      join empty_rows e3

In [0]:
%sql
with 
  empty_rows as (
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n
  ),
  days_after_cancel as (
      select row_number() over(partition by e1.n order by e1.n) days
      from empty_rows e1
      join empty_rows e2
      join empty_rows e3
  ),
  braze as (
      select external_user_id,
             canvas_variation_name,
             min(case when event_name = 'users_messages_email_delivery' then b.dt else null end) as first_email_date,
             min(case when event_name = 'users_canvas_entry' then b.dt else null end) as canvas_entry_date,
             min(case when canvas_variation_name =  'Control' and event_name = 'users_canvas_entry' then b.dt 
                      when canvas_variation_name <> 'Control' and event_name = 'users_messages_email_delivery' then b.dt 
                      else null end) as test_date,
             count(case when b.canvas_step_name like '%10%' then 1 else null end) num_offers_10,
             count(case when b.canvas_step_name like '%20%' then 1 else null end) num_offers_20,
             count(case when b.canvas_step_name like '%30%' then 1 else null end) num_offers_30,
             count(case when b.canvas_step_name like '%40%' then 1 else null end) num_offers_40
      from bronze.braze_events b
      where lower(canvas_name) like '%winback%'
        and event_name in ('users_canvas_entry','users_messages_email_delivery')
      group by 1,2
      having canvas_entry_date >= '2021-01-01'
  ),
  subscriptions as (
      select s.hs_user_id,
             s.voucher_code,
             s.plan_initial_term_months,
             s.dt,
             s.free_trial_start_date,
             s.free_trial_end_date,
             s.paid_subscription_start_date,
             s.paid_subscription_end_date,
             s.created_timestamp,
             s.updated_timestamp,
             lead(s.paid_subscription_start_date) over (partition by s.hs_user_id order by s.paid_subscription_start_date) next_paid_subscription_start_date,
             lead(s.voucher_code) over (partition by hs_user_id order by s.dt, s.created_timestamp, s.updated_timestamp) next_voucher_code
      from silver.fact_subscription s
      where lower(s.country_code) = 'us'
        and lower(s.subscription_type) = 'b2c'
  ),
  all_users as (
      select s.*,
           b.first_email_date,
           b.canvas_variation_name,
           b.num_offers_10,
           b.num_offers_20,
           b.num_offers_30,
           b.num_offers_40,
           b.canvas_entry_date,
           row_number() over(partition by s.hs_user_id order by datediff(b.canvas_entry_date,s.paid_subscription_end_date)) sub_rank
      from braze b
      inner join subscriptions s
        on b.external_user_id = s.hs_user_id
        and b.canvas_entry_date >= s.paid_subscription_end_date
        and s.paid_subscription_end_date between '2021-01-01' and '2022-05-01'
        and datediff(s.paid_subscription_end_date, s.paid_subscription_start_date) >= 28
  ),
tmp as (
    select hs_user_id,
           paid_subscription_end_date,
           next_paid_subscription_start_date,
           voucher_code,
           next_voucher_code,
           first_email_date,
           datediff(first_email_date, paid_subscription_end_date) days_since_cancel,
           case when datediff(coalesce(next_paid_subscription_start_date,'9999-12-31'), first_email_date) <=28 then 1 else 0 end convert_28day_ind
    from all_users a
    where (lower(canvas_variation_name) like '%winback%' or lower(canvas_variation_name) like '%10%' or lower(canvas_variation_name) like '%40%')
      and num_offers_10 + num_offers_20 + num_offers_30 + num_offers_40 > 0
      and sub_rank = 1
      and paid_subscription_end_date is not null
      and coalesce(next_paid_subscription_start_date,'9999-12-31') >= paid_subscription_end_date
  )
select next_voucher_code,
       count(distinct hs_user_id) num_returns
from tmp
group by 1

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

query_1 = f"""
  with 
  empty_rows as (
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n
  ),
  days_after_cancel as (
      select row_number() over(partition by e1.n order by e1.n) days
      from empty_rows e1
      join empty_rows e2
      join empty_rows e3
  ),
  braze as (
      select external_user_id,
             canvas_variation_name,
             min(case when event_name = 'users_messages_email_delivery' then b.dt else null end) as first_email_date,
             min(case when event_name = 'users_canvas_entry' then b.dt else null end) as canvas_entry_date,
             min(case when canvas_variation_name =  'Control' and event_name = 'users_canvas_entry' then b.dt 
                      when canvas_variation_name <> 'Control' and event_name = 'users_messages_email_delivery' then b.dt 
                      else null end) as test_date,
             count(case when b.canvas_step_name like '%10%' then 1 else null end) num_offers_10,
             count(case when b.canvas_step_name like '%20%' then 1 else null end) num_offers_20,
             count(case when b.canvas_step_name like '%30%' then 1 else null end) num_offers_30,
             count(case when b.canvas_step_name like '%40%' then 1 else null end) num_offers_40
      from bronze.braze_events b
      where lower(canvas_name) like '%winback%'
        and event_name in ('users_canvas_entry','users_messages_email_delivery')
      group by 1,2
      having canvas_entry_date >= '2021-01-01'
  ),
  subscriptions as (
      select s.hs_user_id,
             s.voucher_code,
             s.plan_initial_term_months,
             s.dt,
             s.free_trial_start_date,
             s.free_trial_end_date,
             s.paid_subscription_start_date,
             s.paid_subscription_end_date,
             s.created_timestamp,
             s.updated_timestamp,
             lead(s.paid_subscription_start_date) over (partition by s.hs_user_id order by s.paid_subscription_start_date) next_paid_subscription_start_date,
             lead(s.voucher_code) over (partition by hs_user_id order by s.dt, s.created_timestamp, s.updated_timestamp) next_voucher_code
      from silver.fact_subscription s
      where lower(s.country_code) = 'us'
        and lower(s.subscription_type) = 'b2c'
  ),
  all_users as (
      select s.*,
           b.first_email_date,
           b.canvas_variation_name,
           b.num_offers_10,
           b.num_offers_20,
           b.num_offers_30,
           b.num_offers_40,
           b.canvas_entry_date,
           row_number() over(partition by s.hs_user_id order by datediff(b.canvas_entry_date,s.paid_subscription_end_date)) sub_rank
      from braze b
      inner join subscriptions s
        on b.external_user_id = s.hs_user_id
        and b.canvas_entry_date >= s.paid_subscription_end_date
        and s.paid_subscription_end_date between '2021-01-01' and '2022-05-01'
        and datediff(s.paid_subscription_end_date, s.paid_subscription_start_date) >= 28
  )
  --tmp as (
      select hs_user_id,
             paid_subscription_end_date,
             next_paid_subscription_start_date,
             first_email_date,
             datediff(first_email_date, paid_subscription_end_date) days_since_cancel,
             case when lower(coalesce(next_voucher_code,'NA')) like '%winback%' and datediff(coalesce(next_paid_subscription_start_date,'9999-12-31'), first_email_date) <=28 then 1 else 0 end convert_28day_ind
      from all_users a
      where (lower(canvas_variation_name) like '%winback%' or lower(canvas_variation_name) like '%10%' or lower(canvas_variation_name) like '%40%')
        and num_offers_10 + num_offers_20 + num_offers_30 + num_offers_40 > 0
        and sub_rank = 1
        and paid_subscription_end_date is not null
        and coalesce(next_paid_subscription_start_date,'9999-12-31') >= paid_subscription_end_date
  --)
  --select floor((days_since_cancel-1)/10)+1,
  --       avg(convert_28day_ind),
  --       count(distinct hs_user_id) num_users
  --from tmp
  --group by 1
  --order by 1
"""

query_2 = f"""
  with 
  empty_rows as (
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n union all
      select 1 n
  ),
  days_after_cancel as (
      select row_number() over(partition by e1.n order by e1.n) days
      from empty_rows e1
      join empty_rows e2
      join empty_rows e3
  ),
  braze as (
      select external_user_id,
             canvas_variation_name,
             min(case when event_name = 'users_messages_email_delivery' then b.dt else null end) as first_email_date,
             min(case when event_name = 'users_canvas_entry' then b.dt else null end) as canvas_entry_date,
             min(case when canvas_variation_name =  'Control' and event_name = 'users_canvas_entry' then b.dt 
                      when canvas_variation_name <> 'Control' and event_name = 'users_messages_email_delivery' then b.dt 
                      else null end) as test_date,
             count(case when b.canvas_step_name like '%10%' then 1 else null end) num_offers_10,
             count(case when b.canvas_step_name like '%20%' then 1 else null end) num_offers_20,
             count(case when b.canvas_step_name like '%30%' then 1 else null end) num_offers_30,
             count(case when b.canvas_step_name like '%40%' then 1 else null end) num_offers_40
      from bronze.braze_events b
      where lower(canvas_name) like '%winback%'
        and event_name in ('users_canvas_entry','users_messages_email_delivery')
      group by 1,2
      having canvas_entry_date >= '2021-01-01'
  ),
  subscriptions as (
      select s.hs_user_id,
             s.voucher_code,
             s.plan_initial_term_months,
             s.dt,
             s.free_trial_start_date,
             s.free_trial_end_date,
             s.paid_subscription_start_date,
             s.paid_subscription_end_date,
             s.created_timestamp,
             s.updated_timestamp,
             lead(s.paid_subscription_start_date) over (partition by s.hs_user_id order by s.paid_subscription_start_date) next_paid_subscription_start_date,
             lead(s.voucher_code) over (partition by hs_user_id order by s.dt, s.created_timestamp, s.updated_timestamp) next_voucher_code
      from silver.fact_subscription s
      where lower(s.country_code) = 'us'
        and lower(s.subscription_type) = 'b2c'
  ),
  all_users as (
      select s.*,
           b.first_email_date,
           b.canvas_variation_name,
           b.num_offers_10,
           b.num_offers_20,
           b.num_offers_30,
           b.num_offers_40,
           b.canvas_entry_date,
           row_number() over(partition by s.hs_user_id order by datediff(b.canvas_entry_date,s.paid_subscription_end_date)) sub_rank
      from braze b
      inner join subscriptions s
        on b.external_user_id = s.hs_user_id
        and b.canvas_entry_date >= s.paid_subscription_end_date
        and s.paid_subscription_end_date between '2021-01-01' and '2022-05-01'
        and datediff(s.paid_subscription_end_date, s.paid_subscription_start_date) >= 28
  )
  --tmp as (
      select hs_user_id,
             paid_subscription_end_date,
             next_paid_subscription_start_date,
             first_email_date,
             datediff(date_add(paid_subscription_end_date,d.days), paid_subscription_end_date) days_since_cancel,
             case when datediff(coalesce(next_paid_subscription_start_date,'9999-12-31'), date_add(paid_subscription_end_date,d.days)) <=28 then 1 else 0 end convert_28day_ind
      from all_users a
      join days_after_cancel d
        on d.days <= 365
      where lower(canvas_variation_name) like '%control%'
        and num_offers_10 + num_offers_20 + num_offers_30 + num_offers_40 = 0
        and sub_rank = 1
        and paid_subscription_end_date is not null
        and coalesce(next_paid_subscription_start_date,'9999-12-31') >= paid_subscription_end_date
  --)
  --select floor((days_since_cancel-1)/10)+1,
  --       avg(convert_28day_ind),
  --       count(distinct hs_user_id) num_users
  --from tmp
  --group by 1
  --order by 1
"""

df_treatment_group = spark.sql(query_1).toPandas()
df_control_group   = spark.sql(query_2).toPandas()

df_treatment_group = df_treatment_group.loc[(df_treatment_group["days_since_cancel"]>=0) & (df_treatment_group["days_since_cancel"]<=365),:]
df_treatment_group['days_since_cancel_bin'] = pd.cut(df_treatment_group['days_since_cancel'], bins=5)

df_control_group = df_control_group.loc[(df_control_group["days_since_cancel"]>=0) & (df_control_group["days_since_cancel"]<=365),:]
df_control_group['days_since_cancel_bin'] = pd.cut(df_control_group['days_since_cancel'], bins=5)

In [0]:
df_treatment_group['days_since_cancel_bin'] = pd.qcut(df_treatment_group['days_since_cancel'], q=8, duplicates="drop")
df_treatment_group.groupby(by="days_since_cancel_bin").mean("convert_28day_ind").reset_index(0, drop=False).head()

In [0]:
df_control_group['days_since_cancel_bin'] = pd.qcut(df_control_group['days_since_cancel'], q=8, duplicates="drop")
df_control_group.groupby(by="days_since_cancel_bin").mean("convert_28day_ind").reset_index(0, drop=False).head()

In [0]:
# take rate vs. days after cancellation at which email was sent
sns.set(rc={"figure.figsize":(15, 10)})
fig, ax = plt.subplots()
ax = sns.barplot(x="days_since_cancel_bin", 
                 y="convert_28day_ind", 
                 color="b",
                 alpha=0.6,
                 label="treatment",
                 data=df_treatment_group.groupby(by="days_since_cancel_bin").mean("convert_28day_ind").reset_index(0, drop=False))
#ax = sns.barplot(x="days_since_cancel", 
#                 y="convert_28day_ind", 
#                 color="r",
#                 alpha=0.6,
#                 label="control",
#                 data=df_control_group.groupby(by="days_since_cancel_bin").mean("convert_28day_ind").reset_index(0, drop=False))
plt.legend()
plt.show()

In [0]:
# take rate vs. days after cancellation at which email was sent
sns.set(rc={"figure.figsize":(15, 10)})
fig, ax = plt.subplots()
ax = sns.barplot(x="days_since_cancel_bin", 
                 y="convert_28day_ind", 
                 color="r",
                 alpha=0.6,
                 label="control",
                 data=df_control_group.groupby(by="days_since_cancel_bin").mean("convert_28day_ind").reset_index(0, drop=False))
plt.legend()
plt.show()

In [0]:
df_treatment_group[["days_since_cancel"]].head()

In [0]:
# take rate vs. days after cancellation at which email was sent
sns.set(rc={"figure.figsize":(15, 10)})
fig, ax = plt.subplots()
bins = np.histogram_bin_edges(df_treatment_group.loc[(df_treatment_group["days_since_cancel"]<=365),"days_since_cancel"], bins=50)
ax = sns.histplot(x="days_since_cancel", 
                  alpha=0.6,
                  data=df_treatment_group.loc[(df_treatment_group["days_since_cancel"]<=365) & 
                                              (df_treatment_group["convert_28day_ind"]==1)],
                  stat="probability",
                  label="converted",
                  bins=bins
                  #hue="convert_28day_ind"
                 )
ax = sns.histplot(x="days_since_cancel", 
                  alpha=0.4,
                  data=df_treatment_group.loc[(df_treatment_group["days_since_cancel"]<=365) & 
                                              (df_treatment_group["convert_28day_ind"]==0)],
                  stat="probability",
                  label="not converted",
                  color="red",
                  bins=bins
                 )
plt.legend()
plt.show()

In [0]:
bins